In [24]:
import io

In [25]:
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from apiclient import errors
from apiclient import http

In [27]:
def download_file(real_file_id):
    """Downloads a file
    Args:
        real_file_id: ID of the file to download
    Returns : IO object with location.

    Load pre-authorized user credentials from the environment.
    TODO(developer) - See https://developers.google.com/identity
    for guides on implementing OAuth2 for the application.
    """
    creds, _ = google.auth.default()

    try:
        # create drive api client
        service = build('drive', 'v3', credentials=creds)

        file_id = real_file_id

        # pylint: disable=maybe-no-member
        request = service.files().get_media(fileId=file_id)
        file = io.BytesIO()
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(F'Download {int(status.progress() * 100)}.')

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None

    return file.getvalue()

In [13]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly'
          , 'https://www.googleapis.com/auth/drive'
          , 'https://www.googleapis.com/auth/youtube']

In [19]:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=92784570709-gg71nkh6isgt56vag26h3foksfh7k0j7.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53227%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=GzKcsZ03REUH5avM9fy7umMQZpjWvC&access_type=offline


In [52]:
drive = build('drive', 'v3', credentials=creds)
sheet = build('sheets', 'v4', credentials=creds).spreadsheets()

In [ ]:
sheet = build('sheets', 'v4', credentials=creds).spreadsheets()

In [96]:
metadata = drive.files().get(fileId='1hn4sMoIY4t0g68QiUCFL1D7T2AsU0Pjt').execute()

In [83]:
from pathlib import Path

In [100]:
def download_video(drive, file_id):
    metadata = drive.files().get(fileId=file_id).execute()
    name = metadata['name']
    request = drive.files().get_media(fileId=file_id)
    file = io.BytesIO()
    downloader = MediaIoBaseDownload(file, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(F'Download {int(status.progress() * 100)}.')
    Path(f"./video_cache/{name}").write_bytes(file.getbuffer().tobytes())
    # with open(f"./video_cache/{name}", "wb") as outfile:
    #     # Copy the BytesIO stream to the output file
    #     outfile.write(file.getbuffer())

In [58]:
sheet_id = '1c54uBdMLrTcVQfkZOxjsP59ftf6PbGgTv5T9QhU6gUc'
result = sheet.values().get(spreadsheetId=sheet_id,range='D:D').execute()['values']

In [62]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [64]:
result = flatten(result)

In [65]:
list(filter(lambda x: x.startswith('https'), result))

['https://drive.google.com/file/d/17E9Z_rKqHlDoygN1PIZQuigaRREM8rNT/view?usp=sharing',
 'https://drive.google.com/file/d/1hn4sMoIY4t0g68QiUCFL1D7T2AsU0Pjt/view?usp=drivesdk',
 'https://drive.google.com/file/d/17fe1P_QZfXbTlQB889xGXOMednolxbNp/view?usp=sharing',
 'https://drive.google.com/file/d/19EFTPe9AntLX3x69ZY2dTQAMPUK1og3P/view?usp=drivesdk',
 'https://drive.google.com/file/d/1v4wwuHJfohAyITv4dRSFjNqk3YTA_MCg/view?usp=sharing',
 'https://drive.google.com/file/d/1n6uMxdjp3MloaHqXPUYDvBl6X8caGDyY/view?usp=sharing',
 'https://drive.google.com/file/d/12vqF_g3x5lWb20TZJ2KUkTrE9qCrpwiQ/view?usp=sharing',
 'https://drive.google.com/file/d/1zMdrcOjepjn30b4fpfWe0mL7XCvOWFg8/view?usp=sharing',
 'https://drive.google.com/file/d/17mGA-ZeKHOXxlSRrTiBQDjlXf_776BFG/view?usp=sharing',
 'https://drive.google.com/file/d/1ig6OoyjVQhUf9qcS11OwU3D9R-s3jV8q/view?usp=drivesdk',
 'https://drive.google.com/file/d/1DVhTel85El9eF7GyRDaZf7zHqAiAzBYl/view?usp=sharing',
 'https://drive.google.com/file/d/14soTw

In [72]:
urltoid = lambda x: x.split("/")[5]

In [102]:
driveid = urltoid('https://drive.google.com/file/d/14soTw3e8qKwWrRISwIgdrDNnZEStQ-Xe/view?usp=sharing')

In [103]:
download_video(drive, driveid)

Download 14.
Download 29.
Download 43.
Download 58.
Download 72.
Download 87.
Download 100.
